<h1> Create new design ideas based on existing images with Amazon Bedrock </h1>
<br>

<p> Before starting, please make sure this notebook is using <b>conda_python3</b> kernel from the top right! </p>

<p> In this notebook, let's learn how to use Amazon Bedrock to generate creative design ideas </p>

<p> To run this notebook, go to Cell -> Run All. Inspect the output of each cell block. </p>

<b> Please read the following instructions carefully! </b>
    <ul>
    <li>We highly recommend you to run all cells and inspect output rather than running the cells individually to save time as well as avoid any issues.  
    <li>This notebook is for your understanding. Running this notebook is NOT required for proceeding with the next steps of your workshop.
    <li>In case your notebook does not run as expected or if you run into any errors, please proceed with the next steps provided in the Workshop instructions. 
    <li>If you choose to run the notebooks, please read the comments in the markdown and inspect the output of each cell.
    </ul>

<h3> Install required dependencies </h3>
<p> <b>Note:</b> If you notice any ERRORs from the following cell, ignore them and proceed with the next cells.</p><br>

In [ ]:
%pip install --quiet --no-build-isolation --upgrade \
    "boto3==1.28.63" \
    "awscli==1.29.63" \
    "botocore==1.31.63" 

<h3> Import required packages </h3>

In [ ]:
import json
import os
import sys
import boto3
import botocore

# For image operations
from PIL import Image
import base64
import io
import requests

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

<h3> Initialize Bedrock client </h3><br>

In [ ]:
 boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

<p>Create a function to convert an image into a base64 string since Stabile Diffusion Model expects the image to be in base64 string format</p>

In [ ]:
def image_to_base64(img) -> str:
    if isinstance(img, str):
        if os.path.isfile(img):
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")

<h3>Read a product image</h3>

<p> Note: We are using one of the images from the <a href="https://github.com/zalandoresearch/feidegger/tree/master">FEIDEGGER</a> dataset. This is the <a href="https://github.com/zalandoresearch/feidegger/blob/master/LICENSE">license</a> for this dataset. </p>

In [ ]:
image_url = "https://img01.ztat.net/article/spp-media-p1/3c8812d8b6233a55a5da06b19d780302/dc58460c157b426b817f13e7a2f087c5.jpg"

response = requests.get(image_url)
image = Image.open(io.BytesIO(response.content))

resize = image.resize((512,512))

resize.show()

<h4> Convert this image to a base 64 string for Bedrock's Stable Diffusion Model </h4>

In [ ]:
init_image_b64 = image_to_base64(resize)
# print_ww(init_image_b64)

### Image prompts to pass to Bedrock 

In [ ]:
# This prompt is used to generate new ideas from the existing image
change_prompt = "add floral prints to dress"

# Negative prompts that will be given -1.0 weight while generating new image
negative_prompts = ['poorly rendered',
                    'low quality',
                    'disfigured',
                    'disproportional']

### Compose request to pass to Stable Diffusion Model

In [ ]:
sd_request = json.dumps({
                    "text_prompts": (
                        [{"text": change_prompt, "weight": 1.0}]
                        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
                    ),
                    "cfg_scale": 10,
                    "init_image": init_image_b64,
                    "seed": 0,
                    "start_schedule": 0.5,
                    "steps": 30,
                    "style_preset": "photographic",
                    "image_strength":0.5,
                    "denoising_strength": 0.5
                })

### Call Bedrock Stable Diffusion model to generate new image

In [ ]:
response = boto3_bedrock.invoke_model(body=sd_request, modelId="stability.stable-diffusion-xl")

### Render the newly generated image

In [ ]:
response_body = json.loads(response.get('body').read())
genimage_b64_str = response_body["artifacts"][0].get("base64")
genimage = Image.open(io.BytesIO(base64.decodebytes(bytes(genimage_b64_str, "utf-8"))))
genimage.show()

<h3> You've successfully created new design ideas for a product with Amazon Bedrock!</h3>

<p> Please stop the notebook kernel before proceeding. </p>

<h4> Now, let's learn how to integrate Amazon Bedrock into your web application to do the same. Please go back to Workshop Studio and follow the instructions to replicate this code into your Cloud9 environment. </h4>